In [ ]:
#######################################################################
##  Made by: Dr. Keungoui Kim
##  Title: Scitech data import from Dimensions
##  goal : 
##  Data set:  
##  Time Span:
##  Variables
##      Input: 
##      Output: 
##  Time-stamp: #  "Sun Jan 26 01:47:34 2020":  edited by awe kim ; code
##  Notice :
#######################################################################

In [1]:
### Library
import pandas as pd
import numpy as np
import math

### Directory Setting for Export
#dir = "E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/" # Home
dir = "D:/Google Drive(awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/" # HGU

In [2]:
### API log in
import dimcli
dimcli.login(key="792DDFAFCCA7478D8F37159F274A2783",
             endpoint="https://app.dimensions.ai/api/dsl/v2")

dsl = dimcli.Dsl()

Dimcli - Dimensions API Client (v0.9.9.1)
Connected to: <https://app.dimensions.ai/api/dsl/v2> - DSL v2.5
Method: manual login


In [156]:
### Extract patent data
pat_data = dsl.query_iterative("""search patents
                                  where publications is not empty and year = 2022                    
                                  return patents[id+year+priority_year+cpc+publications+publication_ids]""") #  limit 50000

Starting iteration with limit=1000 skip=0 ...
0-1000 / 12332 (10.04s)
1000-2000 / 12332 (5.32s)
2000-3000 / 12332 (4.10s)
3000-4000 / 12332 (4.69s)
4000-5000 / 12332 (4.85s)
5000-6000 / 12332 (6.19s)
6000-7000 / 12332 (4.84s)
7000-8000 / 12332 (4.53s)
8000-9000 / 12332 (4.95s)
9000-10000 / 12332 (5.99s)
10000-11000 / 12332 (8.05s)
11000-12000 / 12332 (3.11s)
12000-12332 / 12332 (2.19s)
===
Records extracted: 12332


In [161]:
pat_data_df = pat_data.as_dataframe()
print(pat_data_df.shape)
print(pat_data_df.id.nunique())

(12332, 6)
12332


In [162]:
pat_data_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+"2022"+".csv", index=False)
pat_data_df.head()

,id,cpc,priority_year,publication_ids,publications,year
0,WO-2022236335-A1,"[A61K38/177, A61P29/00, C07K14/70578, A61P37/0...",2021.0,"[pub.1027342753, pub.1009940936]","[{'doi': '10.1182/blood-2003-06-2031', 'id': '...",2022
1,WO-2022236333-A2,"[A61B3/0016, A61B3/0033]",2021.0,[pub.1065228315],"[{'doi': '10.1364/ol.35.000739', 'id': 'pub.10...",2022
2,WO-2022236331-A1,"[B05B12/18, B05B12/16, B05B14/00, B05B7/0408]",2021.0,[pub.1103595211],"[{'doi': '10.1002/adem.201701084', 'id': 'pub....",2022
3,WO-2022236308-A1,"[A61B5/0042, A61B34/30, G01R33/3806, G01R33/34...",2021.0,[pub.1092392383],"[{'doi': '10.1109/tmag.2017.2751001', 'id': 'p...",2022
4,WO-2022236297-A1,"[C11D3/48, C11D3/0068, C11D3/381, C11D7/40, C1...",2021.0,[pub.1080312358],"[{'doi': '10.1128/jb.96.2.479-486.1968', 'id':...",2022


In [25]:
### Import .csv files
# pat_data_df = pd.read_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_01.csv")

# from ast import literal_eval

# pat_data_df['publication_ids'] = pat_data_df['publication_ids'].apply(literal_eval) #convert to list type

In [163]:
pat_pub_df = pat_data_df[['id','publication_ids']]
pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)
pat_pub_df.head()

,id,publication_ids
0,WO-2022236335-A1,pub.1027342753
1,WO-2022236335-A1,pub.1009940936
2,WO-2022236333-A2,pub.1065228315
3,WO-2022236331-A1,pub.1103595211
4,WO-2022236308-A1,pub.1092392383


In [164]:
### Extract publication_ids
pub_list = pat_pub_df.publication_ids.unique()
pd.DataFrame(pub_list, columns=["publication_ids"]).to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+"2022"+".csv")

## Finalized Version
- Maximum 50,000 records
- dimcli query cannot recognize ''... so I had to write the loop for each jurisdiction --> fixed
- If I could convert dataframe's column into a big list with "" as elements, I would use Version 2.

### Iterative Codes for extracting patent data

In [259]:
years = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009', '2010', '2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'] #, 
offices = ['EP'] # 'WO', 'US', 'EP'

In [302]:
### Create patent data.frame

for year in years:
    for office in offices:
        ### Patent set
        query = """search patents 
                    where publications is not empty and jurisdiction = """+ '''"'''+office+'''"''' +""" and priority_year=""" + year + """ 
                    return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids+associated_grant_ids+funders] sort by priority_date"""
        pat_data = dsl.query_iterative(query) 
        pat_df = pat_data.as_dataframe()

        # Export patent set
        pat_df.to_csv(dir+"patent/pat_data_df_"+year+"_"+office+".csv", index=False)
        
        # Create cpc set
        pat_cpc_df = pat_df[['id','cpc']]
        pat_cpc_df = pat_cpc_df.explode("cpc", ignore_index=True)
        pat_cpc_df.to_csv(dir+"patent/pat_cpc_df_"+year+"_"+office+".csv", index=False)

        if 'associated_grant_ids' in pat_df.columns:
            # Create patent grant set
            pat_grant_list = pat_df[['id','associated_grant_ids']]
            pat_grant_list = pat_grant_list.explode("associated_grant_ids", ignore_index=True)
            pat_grant_list.to_csv(dir+"patent/pat_grant_list_"+year+"_"+office+".csv", index=False)
        
        if 'funders' in pat_df.columns:
            # Create patent funders set (only select cases with funders)
            pat_df = pat_df.dropna(subset=['funders'], how='any', axis=0)
            pat_df.reset_index(drop=True, inplace=True)
            pat_funders_df = pd.DataFrame()
            for i in range(pat_df.shape[0]):
                temp_data = pd.DataFrame.from_records(pat_df['funders'][i])
                temp_data['id'] = pat_df['id'][i]
                pat_funders_df = pd.concat([pat_funders_df, temp_data], axis=0)
            pat_funders_df.to_csv(dir+"patent/pat_funders_df_"+year+"_"+office+".csv", index=False)
    
    print(year, office)

Starting iteration with limit=1000 skip=0 ...
0-1000 / 2493 (2.05s)
1000-2000 / 2493 (2.00s)
2000-2493 / 2493 (1.49s)
===
Records extracted: 2493


2021 EP


### Iterative Codes for extracting publication data

In [260]:
### Create publication data.frame

for year in years:
    for office in offices:
        ### Patent set
        query = """search patents 
                    where publications is not empty and jurisdiction = """+ '''"'''+office+'''"''' +""" and priority_year=""" + year + """ 
                    return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids+associated_grant_ids+funders] sort by priority_date"""
        pat_data = dsl.query_iterative(query) 
        pat_df = pat_data.as_dataframe()

        # Create publication set
        pat_pub_df = pat_df[['id','publication_ids']]
        pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)
        pat_pub_df.to_csv(dir+"pat_pub_df_"+year+"_"+office+".csv", index=False)

        ## Export Publication List version 2. 50,000 each
        pub_list = pat_pub_df.publication_ids.unique()
        pub_list = pd.DataFrame(pub_list, columns=["publication_ids"])
        loop = math.ceil(pub_list.shape[0]/50000)+1
        for i in range(1,loop):
            temp = pub_list.iloc[0:49999,]
            temp.to_csv(dir+"publication/pub_list_"+year+"_"+office+str(i)+".csv", index=False)

            # in operator only can contain maximum 512 elements
            # Thus, I run another loop for extracting 500 publication ids
            inloop = math.ceil(temp.shape[0]/500)+1
            pub_df = pd.DataFrame() # Create empty data.frame  
            for j in range(1, inloop):
                intemp = temp.iloc[0:499,]
                query = """search publications 
                        where id in """+'''['''+','.join(f'"{x}"' for x in intemp['publication_ids'].to_list())+''']'''+"""
                        return publications[id+type+volume+year+issue+title+journal+authors+category_for+supporting_grant_ids]"""
                intemp_data = dsl.query_iterative(query)
                intemp_data = intemp_data.as_dataframe()    
                
                pub_df = pd.concat([pub_df, intemp_data], axis=0, ignore_index=True)
                temp = temp.drop(temp.index[0:499])          

            # Rename pub_id
            pub_df.rename(columns={'id':'pub_id'}, inplace=True)

            if 'supporting_grant_ids' in pub_df.columns:
                # Create publication grant set
                pub_grant_list = pub_df[['pub_id','supporting_grant_ids']]
                pub_grant_list = pub_grant_list.explode("supporting_grant_ids", ignore_index=True)
                pub_grant_list.to_csv(dir+"publication/pub_grant_list_"+year+"_"+office+".csv", index=False)

            if 'category_for' in pub_df.columns:
                # Create publication category set
                pub_category_df = pd.DataFrame()
                for k in range(pub_df.shape[0]):
                    if pd.isna(pub_df['category_for'])[k] == False:
            #        if sum(pd.isna(pub_df['category_for'])[k]) != 0:
                        temp_data = pd.DataFrame.from_records(pub_df['category_for'][k])
                        temp_data['pub_id'] = pub_df['pub_id'][k]
                        pub_category_df = pd.concat([pub_category_df, temp_data], axis=0, ignore_index=True)
                        print(k)
                pub_category_df.to_csv(dir+"publication/pub_category_df_"+year+"_"+office+"_"+str(i)+".csv", index=False)
            
            if 'authors' in pub_df.columns:
                # Create authors set
                pub_authors_df = pd.DataFrame()
                for k in range(pub_df.shape[0]):
                    if pd.isna(pub_df['authors'])[k] == False:
#                    if sum(pd.isna(pub_df['authors'])[k]) != 0: # pd.isna(pub_df['authors'])[k] != True:
                        temp_data = pd.DataFrame.from_records(pub_df['authors'][k])
                        temp_data['pub_id'] = pub_df['pub_id'][k]
                        pub_authors_df = pd.concat([pub_authors_df, temp_data], axis=0, ignore_index=True)
                        print(k)
                pub_authors_df.to_csv(dir+"publication/pub_authors_df_"+year+"_"+office+"_"+str(i)+".csv", index=False)

                if 'affiliations' in pub_authors_df.columns:
                    # Create affiliations set
                    pub_affiliations_df = pd.DataFrame()
                    for k in range(pub_authors_df.shape[0]):
                        if pd.isna(pub_authors_df['affiliations'])[k] == False:
    #                    if sum(pd.isna(pub_df['affiliations'])[k]) != 0: # pd.isna(pub_authors_df['affiliations'])[k] != True:
                            temp_data = pd.DataFrame.from_records(pub_authors_df['affiliations'][k])
                            temp_data['pub_id'] = pub_authors_df['pub_id'][k]
                            pub_affiliations_df = pd.concat([pub_affiliations_df, temp_data], axis=0, ignore_index=True)
                    pub_affiliations_df.to_csv(dir+"publication/pub_affiliations_df_"+year+"_"+office+"_"+str(i)+".csv", index=False)

            # Save publication set
            pub_df.to_csv(dir+"publication/pub_data_df_"+year+"_"+office+"_"+str(i)+".csv", index=False)
            pub_list = pub_list.drop(pub_list.index[0:49999])          

    print(year, office)        

Starting iteration with limit=1000 skip=0 ...
0-1000 / 6576 (1.79s)
1000-2000 / 6576 (4.84s)
2000-3000 / 6576 (6.00s)
3000-4000 / 6576 (1.81s)
4000-5000 / 6576 (2.10s)
5000-6000 / 6576 (2.69s)
6000-6576 / 6576 (1.61s)
===
Records extracted: 6576
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (2.60s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (4.25s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (2.62s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (2.82s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (5.84s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (2.66s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (2.77s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (2.72s)
===
Records extracted: 499
Starting i

In [258]:
pub_grant_list.head()

,pub_id,supporting_grant_ids
0,pub.1008506950,NaN
1,pub.1030162990,NaN
2,pub.1037168643,grant.2519270
3,pub.1037168643,grant.2633999
4,pub.1037168643,grant.2683894


In [255]:
pub_category_df.head()

,id,name,pub_id
0,80001,"30 Agricultural, Veterinary and Food Sciences",pub.1125683875
1,80029,3006 Food Sciences,pub.1125683875
2,80001,"30 Agricultural, Veterinary and Food Sciences",pub.1104426822
3,80029,3006 Food Sciences,pub.1104426822
4,80002,31 Biological Sciences,pub.1025486942


In [257]:
pub_authors_df.head()

,affiliations,corresponding,current_organization_id,first_name,last_name,orcid,raw_affiliation,researcher_id,pub_id
0,"[{'city': 'Multan', 'city_id': 1169825, 'count...",True,grid.413016.1,Haq,Nawaz,[0000-0001-8336-5280],"[Bahauddin Zakariya University, Department of ...",ur.012260462305.70,pub.1125683875
1,"[{'city': 'Multan', 'city_id': 1169825, 'count...",,grid.411501.0,Muhammad Aslam,Shad,None,"[Bahauddin Zakariya University, Department of ...",ur.015543155011.62,pub.1125683875
2,"[{'city': 'Multan', 'city_id': 1169825, 'count...",,,Najiha,Rehman,[],"[Bahauddin Zakariya University, Institute of C...",None,pub.1125683875
3,"[{'city': 'Multan', 'city_id': 1169825, 'count...",,grid.9026.d,Hina,Andaleeb,None,"[Bahauddin Zakariya University, Department of ...",ur.012775222337.16,pub.1125683875
4,"[{'city': 'Multan', 'city_id': 1169825, 'count...",,grid.411501.0,Najeeb,Ullah,[0000-0003-0900-5927],"[Bahauddin Zakariya University, Department of ...",ur.014426322101.31,pub.1125683875


In [256]:
pub_affiliations_df.head()

,city,city_id,country,country_code,id,name,raw_affiliation,state,state_code,pub_id
0,Multan,1169825,Pakistan,PK,grid.411501.0,Bahauddin Zakariya University,"Bahauddin Zakariya University, Department of B...",None,None,pub.1125683875
1,Multan,1169825,Pakistan,PK,grid.411501.0,Bahauddin Zakariya University,"Bahauddin Zakariya University, Department of B...",None,None,pub.1125683875
2,Multan,1169825,Pakistan,PK,grid.411501.0,Bahauddin Zakariya University,"Bahauddin Zakariya University, Institute of Ch...",None,None,pub.1125683875
3,Multan,1169825,Pakistan,PK,grid.411501.0,Bahauddin Zakariya University,"Bahauddin Zakariya University, Department of B...",None,None,pub.1125683875
4,Multan,1169825,Pakistan,PK,grid.411501.0,Bahauddin Zakariya University,"Bahauddin Zakariya University, Department of B...",None,None,pub.1125683875


### Original version

In [ ]:
### Create publication list
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'] 
offices = ['EP']

for year in years:
    for office in offices:
        # Create publication set
        pat_pub_df = pat_df[['id','publication_ids']]
        pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

        ## Export Publication List version 1. ALL
        pub_list = pat_pub_df.publication_ids.unique()
        pub_list = pd.DataFrame(pub_list, columns=["publication_ids"])
        pub_list.to_csv(dir+"pub_list_df_"+year+"_"+office+".csv", index=False)

        ## Export Publication List version 2. 50,000 each
        pub_list = pat_pub_df.publication_ids.unique()
        pub_list = pd.DataFrame(pub_list, columns=["publication_ids"])
        loop = math.ceil(pub_list.shape[0]/50000)+1
        for i in range(1,loop):
            temp = pub_list.iloc[0:49999,]
            temp.to_csv(dir+"pub_list_df_"+year+"_"+office+str(i)+".csv", index=False)
            pub_list = pub_list.drop(pub_list.index[0:49999])          

In [ ]:
### Create publication data.frame
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'] 
offices = ['EP']

for year in years:
    for office in offices:
        ### Publication set

        ## Version 1. Request query for single publication id --> takes longer time
        pub_list = pd.read_csv(dir+"pub_list_df_"+year+"_"+office+".csv")
        # First record
#        query = """search publications 
#                    where id = """+'''"'''+pub_list.publication_ids[0]+'''"'''+"""
#                    return publications[id+type+volume+year+issue+title+journal+authors]"""
#        temp_data = dsl.query(query)    
#        pub_df = temp_data.as_dataframe()

#        for pub in range(1,len(pub_list.publication_ids)):
#            query = """search publications 
#                        where id = """+'''"'''+pub_list.publication_ids[pub]+'''"'''+"""
#                        return publications[id+type+volume+year+issue+title+journal+authors+category_for]"""
#            temp_data = dsl.query(query)    
#            temp_data = temp_data.as_dataframe()
#            
#           # bind rows
#           pub_df = pd.concat([pub_df, temp_data], axis=0)
#
#           print(year, office, pub)

        # Version 2.
        query = """search publications 
                   where id in """+'''['''+','.join(f'"{x}"' for x in pub_list['0'].to_list())+''']'''+"""
                   return publications[id+type+volume+year+issue+title+journal+authors+category_for]"""
        temp_data = dsl.query_iterative(query)
        pub_df = temp_data.as_dataframe()    

        # Save publication set
        pub_df.to_csv(dir+"pub_data_df_"+year+"_"+office+".csv")

### Load Publication list manually


In [ ]:
import os
os.listdir(dir+"/")

### Check Total Counts

In [ ]:
### Check total counts

years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'] 
totalRecUS = []
totalRecEU = []
totalRecWO = []
for year in years:
    ### US
    query = """search patents 
                    where publications is not empty and jurisdiction in ["US"] and priority_year=""" + year + """ 
                    return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query(query) 
    totalRecUS.append(pat_data.count_total)

    ### EP
    query = """search patents 
            where publications is not empty and jurisdiction in ["EP"] and priority_year=""" + year + """ 
            return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query(query) 
    totalRecEUWO.append(pat_data.count_total)

    ### WO
    query = """search patents 
            where publications is not empty and jurisdiction in ["WO"] and priority_year=""" + year + """ 
            return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query(query) 
    totalRecEUWO.append(pat_data.count_total)

    print(year)

# Present result
pd.DataFrame({'year':years,'USRec':totalRecUS,'EUWORec':totalRecEUWO})

#### Alternative codes

In [ ]:
years = '2021'

for year in years:
    query = """search patents 
            where publications is not empty and year=""" + year + """
            return patents[id+year+priority_year+cpc+publications+publication_ids]"""
    pat_data = dsl.query_iterative(query) 
    pat_df = pat_data.as_dataframe()
    rangeIndex = int(temp.count_total/1000)+1

    for t in range(50,rangeIndex):
        query = """search patents 
            where publications is not empty and year=""" + year + """
            return patents[id+year+priority_year+cpc+publications+publication_ids+reference_ids+times_cited] limit 1000 skip """ + str(1000*t)
        temp_data = dsl.query(query)
        temp_df = temp_data.as_dataframe()
        # bind rows
        pat_df = pd.concat([pat_df, temp_df], axis=0)
        print(t)

    # Save patent set
    pat_data_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+year+".csv", index=False)

    # Save publication set
    pat_pub_df = pat_data_df[['id','publication_ids']]
    pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

    # Extract publication_ids
    pub_list = pat_pub_df.publication_ids.unique()
    pd.DataFrame(pub_list, columns=["publication_ids"]).to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+year+".csv", index=False)

In [ ]:
# %%dslloop my_data search publications for "malaria" return publications limit 500

UsageError: %%dslloop is a cell magic, but the cell body is empty. Did you mean the line magic %dslloop (single %)?


In [ ]:
##### EP #####
    query = """search patents 
                where publications is not empty and jurisdiction in ["EP"] and priority_year=""" + year + """ 
                return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query_iterative(query) 
    pat_df = pat_data.as_dataframe()

    # Save patent set
    pat_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+year+"_EP.csv", index=False)

    # Save publication set
    pat_pub_df = pat_df[['id','publication_ids']]
    pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

    # Extract publication_ids
    pub_list = pat_pub_df.publication_ids.unique()
    pub_list = pd.DataFrame(pub_list, columns=["publication_ids"])

    loop = math.ceil(pub_list.shape[0]/50000)+1
    
    for i in range(1,loop):
        temp = pub_list.iloc[0:49999,]
        temp.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+year+"_EP_"+str(i)+".csv")
        pub_list = pub_list.drop(pub_list.index[0:49999])

    ##### WO #####
    query = """search patents 
                where publications is not empty and jurisdiction in ["WO"] and priority_year=""" + year + """ 
                return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query_iterative(query) 
    pat_df = pat_data.as_dataframe()

    # Save patent set
    pat_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+year+"_WO.csv", index=False)

    # Save publication set
    pat_pub_df = pat_df[['id','publication_ids']]
    pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

    # Extract publication_ids
    pub_list = pat_pub_df.publication_ids.unique()
    pub_list = pd.DataFrame(pub_list, columns=["publication_ids"])

    loop = math.ceil(pub_list.shape[0]/50000)+1
    
    for i in range(1,loop):
        temp = pub_list.iloc[0:49999,]
        temp.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+year+"_WO_"+str(i)+".csv")
        pub_list = pub_list.drop(pub_list.index[0:49999])
